# Full Archive Search

- Coppying form this github page https://github.com/twitterdev/Twitter-API-v2-sample-code/blob/master/Full-Archive-Search/full-archive-search.py

In [1]:
import os
from twitter_scraping import twitter_scraper

# Import key from environment variable to keep my credentials secret
bearer_token = os.environ["twitter_bearer_token"]

In [2]:
ts = twitter_scraper(bearer_token)

twitter_scraper v0.1 initialised.


# Turn into function for a given user

Got it to work here, try to functionise it

In [3]:
%%time
ts.get_users_tweets(["nigel_farage"],"2019-10-01T00:00:00.00Z","2019-11-01T00:00:00.00Z")

------------------------------
User: nigel_farage Number: 1 of 1
------------------------------

200
Tweet 0 from 2019-10-31T19:37:00.000Z
CPU times: total: 141 ms
Wall time: 4.67 s


,User,TweetCreated,TweetId
0,nigel_farage,2019-10-31T19:37:00.000Z,1189989669620080641
1,nigel_farage,2019-10-31T18:07:33.000Z,1189967159176388611
2,nigel_farage,2019-10-31T11:08:41.000Z,1189861745009053697
3,nigel_farage,2019-10-29T20:44:06.000Z,1189281779603124227
4,nigel_farage,2019-10-29T18:34:42.000Z,1189249212447215616
...,...,...,...
74,nigel_farage,2019-10-02T15:24:46.000Z,1179416942815465473
75,nigel_farage,2019-10-02T13:42:03.000Z,1179391092057399297
76,nigel_farage,2019-10-02T11:04:54.000Z,1179351544300494850
77,nigel_farage,2019-10-02T10:43:53.000Z,1179346255526404096


In [4]:
%%time
CNNChile_tweets = ts.get_users_tweets(['CNNChile'],"2019-10-01T00:00:00.00Z","2019-10-07T00:00:00.00Z")

------------------------------
User: CNNChile Number: 1 of 1
------------------------------

200
Tweet 0 from 2019-10-06T23:15:00.000Z
200
Tweet 100 from 2019-10-04T16:40:57.000Z
200
Tweet 200 from 2019-10-03T12:08:09.000Z
200
Tweet 300 from 2019-10-01T20:40:50.000Z
CPU times: total: 422 ms
Wall time: 18.3 s


# Chilian News sources

### So far these are the sources that have been identified as potentially interesting:

- El Mercurio (@ElMercurio_cl) / Twitter
- La Tercera (@latercera) / Twitter
- laSegunda (@La_Segunda) / Twitter
- Las Últimas Noticias (@lun) / Twitter
- La Cuarta (@lacuarta) / Twitter
- Meganoticias (@meganoticiascl) / Twitter
- Canal 13 (@canal13) / Twitter
- TVN (@TVN) / Twitter
- 24 Horas (@24HorasTVN) / Twitter
- La Nación Chile (@nacioncl) / Twitter
- Diario Financiero (@DFinanciero) / Twitter
- Chilevisión (@chilevision) / Twitter
- UCV Radio 103.5 FM (@ucvradio) / Twitter
- LaRed (@LaRedTV) / Twitter
- Cooperativa (@Cooperativa) / Twitter
- Radio Pudahuel (@RadioPudahuel) / Twitter
- BioBioChile (@biobio) / Twitter
- El Conquistador 91.3 (@FMConquistador) / Twitter
- EL INFORMADORCHILE🇨🇱 (@INFORMADORCHILE) / Twitter
- CNN Chile (@CNNChile) / Twitter
- CHV Noticias (@CHVNoticias) / Twitter
- El Mostrador (@elmostrador) / Twitter
- Publimetro (@PublimetroChile) / Twitter
- El Desconcierto (@eldesconcierto) / Twitter
- El Dínamo (@el_dinamo) / Twitter
- El Ciudadano (@El_Ciudadano) / Twitter
- El Líbero (@elliberocl) / Twitter
- (4) Diario El Observador (@eo_enlinea) / Twitter




In terms of the time period, if you could look between October 18, 2019 and November 18, 2019, that would be some cool gravy.




In [5]:
%%time

chile_news_users = [
    'ElMercurio_cl',
    'latercera',
    'La_Segunda',
    'lun',
    'lacuarta',
    'meganoticiascl',
    'canal13',
    'TVN',
    '24HorasTVN',
    'nacioncl',
    'DFinanciero',
    'chilevision',
    'ucvradio',
    'LaRedTV',
    'Cooperativa',
    'RadioPudahuel',
    'biobio',
    'FMConquistador',
    'INFORMADORCHILE',
    'CNNChile',
    'CHVNoticias',
    'elmostrador',
    'PublimetroChile',
    'eldesconcierto',
    'el_dinamo',
    'El_Ciudadano',
    'elliberocl',
    'eo_enlinea'
]


start_date = "2019-10-01T00:00:00.00Z"
end_date = "2019-11-01T00:00:00.00Z"

CPU times: total: 0 ns
Wall time: 0 ns


In [6]:
start_date = "2019-10-01T00:00:00.00Z"
end_date = "2019-11-01T00:00:00.00Z"

# Try getting just the dates we want in 2019
results = ts.get_users_tweets(chile_news_users, start_date, end_date)

------------------------------
User: ElMercurio_cl Number: 1 of 28
------------------------------

200
Tweet 0 from 2019-10-31T23:45:00.000Z
200
Tweet 100 from 2019-10-30T12:10:00.000Z
200
Tweet 200 from 2019-10-28T13:20:00.000Z
200
Tweet 300 from 2019-10-26T22:45:00.000Z
200
Tweet 400 from 2019-10-24T14:30:00.000Z
200
Tweet 500 from 2019-10-21T23:40:00.000Z
200
Tweet 600 from 2019-10-19T16:00:00.000Z
200
Tweet 700 from 2019-10-17T21:00:00.000Z
200
Tweet 800 from 2019-10-15T22:00:00.000Z
200
Tweet 900 from 2019-10-14T01:00:00.000Z
200
Tweet 1000 from 2019-10-11T22:00:00.000Z
200
Tweet 1100 from 2019-10-09T23:20:00.000Z
200
Tweet 1200 from 2019-10-08T00:45:00.000Z
200
Tweet 1300 from 2019-10-06T10:50:00.000Z
200
Tweet 1400 from 2019-10-03T23:40:00.000Z
200
Tweet 1500 from 2019-10-01T23:00:00.000Z
------------------------------
User: latercera Number: 2 of 28
------------------------------

200
Tweet 0 from 2019-10-31T23:50:00.000Z
200
Tweet 100 from 2019-10-31T11:20:00.000Z
200
Tweet 20

In [ ]:
results

,User,TweetCreated,TweetId
0,ElMercurio_cl,2019-10-31T23:45:00.000Z,1190052078351273984
1,ElMercurio_cl,2019-10-31T23:30:00.000Z,1190048304261947392
2,ElMercurio_cl,2019-10-31T23:15:00.000Z,1190044528520155146
3,ElMercurio_cl,2019-10-31T23:00:00.000Z,1190040753625387009
4,ElMercurio_cl,2019-10-31T22:45:00.000Z,1190036979003674629
...,...,...,...
62333,eo_enlinea,2019-10-01T13:31:45.000Z,1179026113860902913
62334,eo_enlinea,2019-10-01T13:15:09.000Z,1179021934601920512
62335,eo_enlinea,2019-10-01T13:01:59.000Z,1179018624062894080
62336,eo_enlinea,2019-10-01T12:55:34.000Z,1179017007532036098


In [ ]:
results.to_csv('chile_news_2019_10_01_to_2019_11_01.csv')

In [ ]:
results['User'].value_counts()

Cooperativa        12032
latercera           5779
24HorasTVN          5414
meganoticiascl      4837
PublimetroChile     4576
biobio              4186
DFinanciero         2232
CNNChile            2152
elmostrador         2083
eldesconcierto      2031
El_Ciudadano        1886
lacuarta            1627
ElMercurio_cl       1605
CHVNoticias         1532
el_dinamo           1408
TVN                 1371
canal13             1309
elliberocl          1167
nacioncl            1060
LaRedTV              757
ucvradio             692
chilevision          641
lun                  574
RadioPudahuel        446
eo_enlinea           381
FMConquistador       366
INFORMADORCHILE      194
Name: User, dtype: int64